In [1]:
import sys
from lxml import html as htmlRenderer
import requests
import json
from datetime import date
from datetime import datetime
from random import *

In [2]:

urlInfoComments = "https://comments.eu1.gigya.com/comments.getStreamInfo"
urlGetComments = "https://comments.eu1.gigya.com/comments.getComments"

In [3]:
def extractComments(commentsList=None, urlNoticia=None, specialCase=None):
    print(" \t -> parsing comments list with -{}- elements:".format(len(commentsList)))
    parsedComments = []
    listToParse = commentsList[1] if specialCase else commentsList
    for commentObj in listToParse:
        fechaObj = datetime.fromtimestamp(round(commentObj["timestamp"] / 1000))
        fechaStr = fechaObj.strftime("%d/%m/%Y-%H:%M:%S")
        fechaArr = fechaStr.split("-")
        parsedComment = {
            "urlNoticia": urlNoticia,
            "fecha": fechaArr[0],
            "hora": fechaArr[1],
            "user": commentObj['sender']['name'],
            "commentario": commentObj['commentText'],
            "negVotes": commentObj["TotalVotes"] - commentObj["posVotes"],
            "posVotes": commentObj["posVotes"]
        }
        parsedComments.append(parsedComment)
        if commentObj.get("replies") != None:
            parsedComments = parsedComments + self.extractComments(commentObj.get("replies"), urlNoticia)
    return parsedComments

In [4]:
def lookupForComments(renderedPageHtml=None, url=None):
    streamIDStr = ""

    pageComments = []
    if ".es/noticia/" in url:
        streamIDStr = "noticia-{}".format(url.split("/noticia/")[1].split("/")[0])
    elif "/noticia/" in url:
        auxSplitted = url.split("/noticia/")[1].split("/")[0].split("-")
        streamIDStr = "noticia-{}".format(auxSplitted[len(auxSplitted) - 1])

    if (streamIDStr != ""):
        commentElValStreamId = streamIDStr
        print(" \t-> comment-stream-id to get comments is: {}".format(commentElValStreamId))

        infoArg = {
            "categoryID": "prod",
            "streamID": commentElValStreamId,
            "ctag": "comments_v2",
            "APIKey": "3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH",
            "sourceData": {"categoryID": "prod", "streamID": commentElValStreamId},
            "pageURL": url,
            "cid": "",
            "source": "showCommentsUI",
            "sdk": "js_latest",
            "authMode": "cookie",
            "format": "json",
            "callback": "gigya.callback"
        }
        responseInfoComments = requests.get(urlInfoComments, infoArg)
        infoComments = json.loads(responseInfoComments.text)
        print(" \t-> getting information for article: {}".format(url))
        print(
            " \t-> total of comments for current article: {}".format(infoComments["streamInfo"]["commentCount"]))
        if infoComments["streamInfo"]["commentCount"] > 0:
            # La info dice que hay comentarios, se procede a obtenerlos
            print(" -> total of comments is greater than 0")
            nextTs = ""
            iterate = True
            while (iterate):
                commentsArgs = {
                    "categoryID": "prod",
                    "streamID": commentElValStreamId,
                    "sourceData": {"categoryID": "prod", "streamID": commentElValStreamId},
                    "pageURL": url,
                    "includeSettings": "true",
                    "threaded": "true",
                    "includeUserOptions": "true",
                    "includeUserHighlighting": "true",
                    "lang": "es",
                    "ctag": "comments_v2",
                    "APIKey": "3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH",
                    "cid": "",
                    "source": "showCommentsUI",
                    "sdk": "js_latest",
                    "authMode": "cookie",
                    "format": "json",
                    "callback": "gigya.callback"
                }
                if nextTs != "":
                    commentsArgs.update({
                        "start": nextTs
                    })
                responseComments = requests.get(urlGetComments, commentsArgs)
                commentsResponse = json.loads(responseComments.text)
                iterate = commentsResponse["hasMore"]
                nextTs = commentsResponse["next"]
                pageComments = pageComments + extractComments(commentsResponse["comments"], url)
            print(" \t -> retrieved total of {} comments".format(len(pageComments)))
            print(
                "#############################################################################################")
    return pageComments

# Get IdNoticia

In [5]:
url="https://www.20minutos.es/noticia/3527992/0/lluvia-cuadrantidas-donde-cuando-verlas/"
renderUrl = requests.get(url)
renderedPage = htmlRenderer.fromstring(renderUrl.text)
comments = lookupForComments(renderedPage, url)

 	-> comment-stream-id to get comments is: noticia-3527992
 	-> getting information for article: https://www.20minutos.es/noticia/3527992/0/lluvia-cuadrantidas-donde-cuando-verlas/
 	-> total of comments for current article: 1
 -> total of comments is greater than 0
 	 -> parsing comments list with -1- elements:
 	 -> retrieved total of 1 comments
#############################################################################################


In [6]:
comments

[{'commentario': 'Perforan más de un kilómetro de hielo para llegar a un lago antártico en busca de vida<div><br /></div><div><span>Publicado: 1 ene 2019\xa0</span><br /></div><div><span><br /></span></div><div><span>Los expertos señalan que cualquier vida capaz de soportar las condiciones del lago podría generar esperanzas de encontrar organismos similares en el interior de Marte o en las lunas cubiertas de hielo de Júpiter y Saturno.</span><span><br /></span></div><div><span><br /></span></div><div><span>Los\xa0investigadores del\xa0Acceso Científico a los Lagos Subglacial Antárticos (SALSA, por sus siglas en inglés) han logrado acceder a un lago enterrado bajo más de 325 metros cuadrados de hielo en la Antártida en un intento por descubrir si en ese inhóspito entorno existe vida,\xa0</span>informa<span>\xa0la página web de la organización. El\xa0lago mide casi\xa0</span><span>160 kilómetros cuadrados</span><span>, y fue descubierto hace más de una década a través de imágenes satelit